# Advanced Querying Mongo

Importing libraries and setting up connection

In [ ]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [ ]:
client.list_database_names()

In [ ]:
db = client.get_database("lab")

In [ ]:
db.list_collection_names()

In [ ]:
companies = db.get_collection("oficinas")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [ ]:
proj = {"_id": 0, "name": 1}
name_companies = list(companies.find({"name": "Babelgum"}, proj))
name_companies

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [ ]:
employees_companies = list(companies.find({"number_of_employees": {"$gt": 5000}}).sort("number_of_employees", 1).limit(20))
employees_companies

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [ ]:
foundation_proj = {"_id": 0, "name": 1, "founded_year": 1}
foundation_companies = list(companies.find({"founded_year": {"$gte": 2000}, "founded_year": {"$lte": 2005}}, foundation_proj))

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [ ]:
valam_proj = {"_id": 0, "name": 1, "ipo": 1}
valam_companies = list(companies.find({"ipo.valuation_amount": {"$gt": 100000000}, "founded_year": {"$lt": 2010}}, valam_proj))

In [62]:
valam_companies

[{'ipo': {'pub_day': 18,
   'pub_month': 5,
   'pub_year': 2012,
   'stock_symbol': 'NASDAQ:FB',
   'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD'},
  'name': 'Facebook'},
 {'ipo': {'pub_day': 7,
   'pub_month': 11,
   'pub_year': 2013,
   'stock_symbol': 'NYSE:TWTR',
   'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD'},
  'name': 'Twitter'},
 {'ipo': {'pub_day': 2,
   'pub_month': 3,
   'pub_year': 2012,
   'stock_symbol': 'NYSE:YELP',
   'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD'},
  'name': 'Yelp'},
 {'ipo': {'pub_day': 20,
   'pub_month': 7,
   'pub_year': 2011,
   'stock_symbol': 'NYSE:LNKD',
   'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD'},
  'name': 'LinkedIn'},
 {'ipo': {'pub_day': None,
   'pub_month': 5,
   'pub_year': 1997,
   'stock_symbol': 'NASDAQ:AMZN',
   'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD'},
  'name': 'Amazon'},
 {'ipo': {'pub_day': 17,
   'pub_m

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
small_companies = list(companies.find({"number_of_employees": {"$lt": 1000}, "founded_year": {"$lt": 2005}}).sort("number_of_employees", 1).limit(10))
small_companies

### 6. All the companies that don't include the `partners` field.

In [61]:
lista = ["None"]
not_partners_companies = list(companies.find({"partners": {"$nin": lista}}))
len(not_partners_companies)

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [65]:
lista = ["None"]
code_companies = list(companies.find({"category_code": {"$nin": lista}}))
len(code_companies)

18801

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
# Your Code

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code